# Cross Sectional Systematic risk measure

### Definitions, Pseudocode and python function
**1. CoVaR (Conditional Value at Risk)**

Definition:
CoVaR measures the Value at Risk (VaR) of the financial system, conditional on a particular institution being in distress. It helps assess how much systemic risk a firm contributes.



In [ ]:
Function CoVaR(institution_returns, system_returns, quantile):
    Compute VaR of the institution at the given quantile
    Select system returns where the institution is in distress (below its VaR)
    Compute the system's VaR over those conditional periods
    Return the conditional VaR (CoVaR)

In [1]:
def CoVaR(institution_returns, system_returns, quantile=0.05):
    var_i = np.quantile(institution_returns, quantile)
    mask = institution_returns <= var_i
    covar = np.quantile(system_returns[mask], quantile)
    return covar

**2. MES (Marginal Expected Shortfall)**

Definition:
MES is the average loss of an institution when the market is experiencing extreme negative returns. It shows how much a firm is expected to lose in systemic crisis scenarios.

In [ ]:
Function MES(institution_returns, market_returns, quantile):
    Find the threshold return defining market distress (VaR of the market)
    Identify periods when market returns are below that threshold
    Calculate the average return of the institution during those periods
    Return the average as MES

In [2]:
def MES(institution_returns, market_returns, quantile=0.05):
    threshold = np.quantile(market_returns, quantile)
    stress_periods = market_returns <= threshold
    mes = institution_returns[stress_periods].mean()
    return mes

**3. ΔCoVaR (Delta CoVaR)**

Definition:
ΔCoVaR measures the increase in systemic risk when an institution moves from a normal state (e.g., median) to a stressed state. It quantifies the marginal contribution to systemic risk.

In [ ]:
Function delta_CoVaR(institution_returns, system_returns, quantile):
    Compute institution's VaR at given quantile (distress)
    Compute institution's VaR at 50% quantile (median)
    
    Select system returns when institution is in distress
    Select system returns when institution is in median state
    
    Compute system VaR under both conditions
    Subtract: CoVaR (distress) - CoVaR (median)
    Return the difference as delta CoVaR

In [3]:
def delta_CoVaR(institution_returns, system_returns, quantile=0.05):
    var_i = np.quantile(institution_returns, quantile)
    var_median = np.quantile(institution_returns, 0.5)

    covar_distress = np.quantile(system_returns[institution_returns <= var_i], quantile)
    covar_median = np.quantile(system_returns[institution_returns <= var_median], quantile)
    
    return covar_distress - covar_median

# Combined Python code

In [4]:
import numpy as np

def CoVaR(institution_returns, system_returns, quantile=0.05):
    """
    Estimate CoVaR of the system given an institution is in distress.
    
    Params:
    - institution_returns: np.array of returns for the institution
    - system_returns: np.array of returns for the system or aggregate
    - quantile: quantile level (e.g., 0.05 for 5% VaR)
    
    Returns:
    - CoVaR value
    """
    # Get the threshold where the institution is in distress (its VaR)
    var_i = np.quantile(institution_returns, quantile)
    
    # Conditional returns of system when institution is in distress
    mask = institution_returns <= var_i
    covar = np.quantile(system_returns[mask], quantile)
    
    return covar

def MES(institution_returns, market_returns, quantile=0.05):
    """
    Calculate Marginal Expected Shortfall (MES) for an institution.
    
    Params:
    - institution_returns: np.array
    - market_returns: np.array (e.g., Nifty, S&P500)
    - quantile: tail quantile to define market stress
    
    Returns:
    - MES value
    """
    # Define market distress period (e.g., bottom 5%)
    threshold = np.quantile(market_returns, quantile)
    stress_periods = market_returns <= threshold
    
    # Average return of the institution during those stress periods
    mes = institution_returns[stress_periods].mean()
    return mes

def delta_CoVaR(institution_returns, system_returns, quantile=0.05):
    """
    Calculate Delta CoVaR (change in system risk due to institution's distress)
    
    Params:
    - institution_returns: np.array
    - system_returns: np.array
    - quantile: distress quantile
    
    Returns:
    - delta CoVaR value
    """
    var_i = np.quantile(institution_returns, quantile)
    var_median = np.quantile(institution_returns, 0.5)
    
    # CoVaR when institution is in distress
    covar_distress = np.quantile(system_returns[institution_returns <= var_i], quantile)
    
    # CoVaR when institution is in median state
    covar_median = np.quantile(system_returns[institution_returns <= var_median], quantile)
    
    delta_covar = covar_distress - covar_median
    return delta_covar



In [5]:
##### sample test ######
# Example data
np.random.seed(0)
institution = np.random.normal(-0.01, 0.02, 1000)
system = np.random.normal(-0.005, 0.015, 1000)

# Calculate measures
print("CoVaR:", CoVaR(institution, system))
print("MES:", MES(institution, system))
print("Delta CoVaR:", delta_CoVaR(institution, system))

CoVaR: -0.02813119826436948
MES: -0.006068953998049488
Delta CoVaR: -0.0008824113780089016
